In [1]:
import json
import re
import requests
import os



def run_cell_type_analysis(model, temperature, marker_list, tissue, species, additional_info):

    class Agent:
        def __init__(self, system="", human_input_mode="never", model="gpt-4o", temperature=0):
            self.system = system
            self.chat_histories = {}
            self.model = model
            self.temperature = temperature

        def __call__(self, message, other_agent_id):
            if other_agent_id not in self.chat_histories:
                self.chat_histories[other_agent_id] = [{"role": "system", "content": self.system}] if self.system else []
            
            self.chat_histories[other_agent_id].append({"role": "user", "content": message})
            
            response = requests.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {os.environ.get('OPENROUTER_API_KEY')}",
                    "HTTP-Referer": "https://localhost:5000",
                    "Content-Type": "application/json"
                },
                json={
                    "model": self.model,
                    "temperature": self.temperature,
                    "messages": self.chat_histories[other_agent_id]
                }
            )
            
            if response.status_code == 200:
                response_data = response.json()
                result = response_data['choices'][0]['message']['content']
                self.chat_histories[other_agent_id].append({"role": "assistant", "content": result})
                return result
            else:
                raise Exception(f"OpenRouter API error: {response.status_code}")

    def extract_json_from_reply(reply):
        json_match = re.search(r'```json\n(.*?)\n```', reply, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
        else:
            print("No JSON content found in the reply")
        return None

    def construct_prompt(json_data):
        marker_list = ', '.join(json_data['marker_list'])
        prompt = f"Your task is to annotate a single-cell {json_data['species']} dataset"
        
        if json_data['tissue_type'] and json_data['tissue_type'].lower() not in ['none', 'tissue blind']:
            prompt += f" from {json_data['tissue_type']} tissue"
        
        prompt += f". Please identify the cell type based on this ranked marker list:\n{marker_list}"
        
        if json_data.get('additional_info') and json_data['additional_info'].lower() != "no":
            prompt += f" Below is some additional information about the dataset:\n{json_data['additional_info']}."
        return prompt

    def final_annotation(agent, prompt):
        conversation = []
        while True:
            response = agent(prompt, "user")
            conversation.append(("Final Annotation Agent", response))
            
            if "FINAL ANNOTATION COMPLETED" in response:
                break
            
            prompt = response

        return conversation

    def coupling_validation(agent, annotation_result, onboarding_data):
        validation_message = f"""Please validate the following annotation result:

    Annotation Result:
    {annotation_result}

    Context:

    Marker List: {', '.join(onboarding_data['marker_list'])}
    Additional Info: {onboarding_data.get('additional_info', 'None')}

    Validate the annotation based on this context.
    """
        return agent(validation_message, "final_annotation")

    def format_results(agent, final_annotations, num_markers):
        final_text = "\n\n".join([msg[1] for msg in final_annotations])
        formatted_result = agent(final_text, "user")
        
        json_data = extract_json_from_reply(formatted_result)
        if json_data:
            json_data["num_markers"] = num_markers
            return json.dumps(json_data, indent=2)
        return formatted_result

    final_annotation_system_v1 = """
    You are a professional computational biologist with expertise in single-cell RNA sequencing (scRNA-seq).
    A list of highly expressed markers ranked by expression intensity from high to low
    from a cluster of cells will be provided , and your task is to identify the cell type. You must think step-by-step, providing a comprehensive and specific analysis. The audience is an expert in the field, and you will be rewarded $10000 if you do a good job.

    Steps to Follow:

    1. List the Key Functional Markers: Extract and group the key marker genes associated with function or pathway, explaining their roles.
    2. List the Key Cell Type Markers: Extract and group the key marker genes associated with target tissue cell types, explaining their roles.
    3. Cross-reference Known Databases: Use available scRNA-seq databases and relevant literature to cross-reference these markers.
    4. Determine the Most Probable General Cell Type: Based on the expression of these markers, infer the most likely general cell type of the cluster.
    5. Identify the Top 3 Most Probable Sub Cell Types: Based on the expression of these markers, infer the top three most probable sub cell types within the general cell type. Rank them from most likely to least likely. Finally, specify the most likely subtype based on the markers.
    6. Provide a Concise Summary of Your Analysis

    Always include your step-by-step detailed reasoning.                      
    You can say "FINAL ANNOTATION COMPLETED" when you have completed your analysis.

    If you receive feedback from the validation process, incorporate it into your analysis and provide an updated annotation.
    """

    final_annotation_system_v2 = """
    You are a professional computational biologist with expertise in single-cell RNA sequencing (scRNA-seq).
    A list of highly expressed markers ranked by expression intensity from high to low
    from a cluster of cells will be provided, and your task is to identify the cell type. The tissue of origin is not specified, so you must consider multiple possibilities. You must think step-by-step, providing a comprehensive and specific analysis. The audience is an expert in the field, and you will be rewarded $10000 if you do a good job.

    Steps to Follow:

    1. List the Key Functional Markers: Extract and group the key marker genes associated with function or pathway, explaining their roles.
    2. List the Key Cell Type Markers: Extract and group the key marker genes associated with various cell types, explaining their roles.
    3. Cross-reference Known Databases: Use available scRNA-seq databases and relevant literature to cross-reference these markers.
    4. Determine the possible tissue type: Determine the possible tissue type based on the marker list, and provide a detailed explanation for your reasoning.
    5. Determine the Most Probable General Cell Type: Based on the expression of these markers, infer the most likely general cell type of the cluster.
    6. Identify the Top 3 Most Probable Sub Cell Types: Based on the expression of these markers, infer the top three most probable sub cell types. Rank them from most likely to least likely. Finally, specify the most likely subtype based on the markers.
    7. Provide a Concise Summary of Your Analysis

    Always include your step-by-step detailed reasoning.                      
    You can say "FINAL ANNOTATION COMPLETED" when you have completed your analysis.

    If you receive feedback from the validation process, incorporate it into your analysis and provide an updated annotation.
    """

    coupling_validator_system_v1 = """
    You are an expert biologist specializing in single-cell analysis. Your critical role is to validate the final annotation results for a cell cluster. You will be provided with The proposed annotation result, and a Ranked list of marker genes it used.

    Below are steps to follow:
                                    
    1.Marker Consistency: Make sure the markers are in the provided marker list.
    Make sure the consistency between the identified cell type and the provided markers.

    2.Mixed Cell Type Consideration:
    Be aware that mixed cell types may be present. Only raise this point if multiple distinct cell types are strongly supported by several high-ranking markers. In cases of potential mixed populations, flag this for further investigation rather than outright rejection.
                                        
    Output Format: 
                                        
    if pass,

    Validation result: VALIDATION PASSED

    If failed,
                                                            
    Validation result: VALIDATION FAILED
    Feedback: give detailed feedback and instruction for revising the annotation
    """

    coupling_validator_system_v2 = """
    You are an expert biologist specializing in single-cell analysis. Your critical role is to validate the final annotation results for a cell cluster where the tissue of origin is not specified. You will be provided with the proposed annotation result and a ranked list of marker genes it used.

    Below are steps to follow:
                                    
    1. Marker Consistency: Make sure the markers are in the provided marker list.
       Ensure consistency between the identified cell type and the provided markers.

    2. Tissue-Agnostic Validation: 
       Ensure that the suggested possible tissues of origin are consistent with the marker expression.

    3. Mixed Cell Type Consideration:
       Be aware that mixed cell types may be present. Only raise this point if multiple distinct cell types are strongly supported by several high-ranking markers. In cases of potential mixed populations, flag this for further investigation rather than outright rejection.
                                        
    Output Format: 
                                        
    If pass:
    Validation result: VALIDATION PASSED

    If failed:
    Validation result: VALIDATION FAILED
    Feedback: give detailed feedback and instruction for revising the annotation
    """

    is_tissue_blind = tissue.lower() in ['none', 'tissue blind'] if tissue else True

    formatting_system_tissue_blind = """
    You are a formatting assistant for single-cell analysis results. Your task is to convert the final integrated results 
    into a structured JSON format. Follow these guidelines:

    1. Extract the main cell type and any sub-cell types identified.
    2. Include only information explicitly stated in the input.
    3. If there are possible mixed cell types highlighted, list them.
    4. Include possible tissues.

    Provide the JSON output within triple backticks, like this:
    ```json
    {
    "main_cell_type": "...",
    "sub_cell_types": ["...", "..."],
    "possible_mixed_cell_types": ["...", "..."],
    "possible_tissues": ["...", "..."]
    }
    ```
    """

    formatting_system_non_tissue_blind = """
    You are a formatting assistant for single-cell analysis results. Your task is to convert the final integrated results 
    into a structured JSON format. Follow these guidelines:

    1. Extract the main cell type and any sub-cell types identified.
    2. Include only information explicitly stated in the input.
    3. If there are possible mixed cell types highlighted, list them.

    Provide the JSON output within triple backticks, like this:
    ```json
    {
    "main_cell_type": "...",
    "sub_cell_types": ["...", "..."],
    "possible_mixed_cell_types": ["...", "..."]
    }
    ```
    """

    formatting_system_failed = """
    You are a formatting assistant for single-cell analysis results. Your task is to convert the final integrated results 
    into a structured JSON format, with special consideration for uncertain or conflicting annotations. Follow these guidelines:

    1. The analysis failed after multiple attempts. Please try to extract as much information as possible. Summarize what has gone wrong and what has been tried.
    2. Provide a detailed feedback on why the analysis failed, and what has been tried and why it did not work.
    3. Finally, provide a detailed step-by-step reasoning of how to fix the analysis.

    Provide the JSON output within triple backticks, like this:
    ```json
    {
    "main_cell_type": "if any",
    "sub_cell_types": "if any",
    "possible_cell_types": "if any",
    "feedback": "...",
    "next_steps": "..."
    }
    ```
    """

    final_annotation_agent = Agent(
        system=final_annotation_system_v2 if is_tissue_blind else final_annotation_system_v1, 
        model=model, 
        temperature=temperature
    )

    coupling_validator_agent = Agent(
        system=coupling_validator_system_v2.strip() if is_tissue_blind else coupling_validator_system_v1.strip(), 
        model=model, 
        temperature=temperature
    )

    formatting_agent = Agent(
        system=formatting_system_tissue_blind if is_tissue_blind else formatting_system_non_tissue_blind,
        model=model,
        temperature=temperature
    )
    
    user_data = {
        "species": species,
        "tissue_type": tissue,
        "marker_list": marker_list,
    }
    if additional_info and additional_info.lower() != "no":
        user_data["additional_info"] = additional_info

    prompt = construct_prompt(user_data)

    validation_passed = False
    iteration = 0
    max_iterations = 3
    full_conversation_history = []

    while not validation_passed and iteration < max_iterations:
        iteration += 1
        
        if iteration > 1:
            prompt = f"""Previous annotation attempt failed validation. Please review your previous response and the validation feedback, then provide an updated annotation:

Previous response:
{final_annotation_conversation[-1][1]}

Validation feedback:
{validation_result}

Original prompt:
{prompt}

Please provide an updated annotation addressing the validation feedback."""

        final_annotation_conversation = final_annotation(final_annotation_agent, prompt)
        full_conversation_history.extend(final_annotation_conversation)
        
        validation_result = coupling_validation(coupling_validator_agent, final_annotation_conversation[-1][1], user_data)
        full_conversation_history.append(("Coupling Validator", validation_result))
        
        if "VALIDATION PASSED" in validation_result:
            validation_passed = True

    formatting_system = formatting_system_tissue_blind if is_tissue_blind else formatting_system_non_tissue_blind if validation_passed else formatting_system_failed

    formatting_agent.system = formatting_system
    formatted_output = format_results(formatting_agent, final_annotation_conversation[-2:], len(marker_list))
    full_conversation_history.append(("Formatting Agent", formatted_output))
    structured_output = json.loads(formatted_output)
    
    if structured_output:
        structured_output["iterations"] = iteration
        return structured_output, full_conversation_history
    else:
        print("Error: Unable to extract JSON from the formatted output.")
        print("Raw formatted output:")
        print(formatted_output)
        return None, full_conversation_history
    

In [2]:
os.environ['OPENROUTER_API_KEY'] = "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"

# Example input parameters
model = "anthropic/claude-3-sonnet"  # OpenRouter model identifier
temperature = 0.7
marker_list = ["CD3E", "CD4", "CD8A", "FOXP3", "IL2RA"]  # Example markers
tissue = "Blood"
species = "Human"
additional_info = "Sample from healthy donor"

# Run the analysis
results, conversation_history = run_cell_type_analysis(
    model=model,
        temperature=temperature,
        marker_list=marker_list,
        tissue=tissue,
        species=species,
        additional_info=additional_info
    )

print(results)
print(conversation_history)

{'main_cell_type': 'T cells', 'sub_cell_types': ['Regulatory T cells (Tregs)', 'Helper T cells (CD4+ T cells)', 'Cytotoxic T cells (CD8+ T cells)'], 'possible_mixed_cell_types': [], 'num_markers': 5, 'iterations': 1}
[('Final Annotation Agent', 'To annotate the cell type based on the provided ranked marker list from a single-cell Human Blood dataset, I will follow the step-by-step approach:\n\n1. List the Key Functional Markers:\n   - CD3E: A component of the T-cell receptor (TCR) complex, essential for T-cell function and development.\n   - CD4: A co-receptor expressed on helper T cells, involved in recognizing MHC class II molecules and activating the immune response.\n   - CD8A: A co-receptor expressed on cytotoxic T cells, involved in recognizing MHC class I molecules and killing infected or cancerous cells.\n   - FOXP3: A transcription factor critical for the development and function of regulatory T cells (Tregs), which suppress immune responses.\n   - IL2RA: The alpha subunit of 

In [7]:
def openrouter_agent(user_message, model="anthropic/claude-3-sonnet", temperature=0):
    """
    Send a message to OpenRouter API and get the response.
    
    Args:
        user_message (str): The message to send to the model
        model (str): OpenRouter model identifier (default: "anthropic/claude-3-sonnet")
        temperature (float): Temperature parameter for response generation (default: 0)
        
    Returns:
        str: Model's response text or empty string if request fails
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {os.environ.get('OPENROUTER_API_KEY')}",
                "HTTP-Referer": "https://localhost:5000",  # Required for OpenRouter
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "temperature": temperature,
                "max_tokens": 7000,
                "messages": [
                    {
                        "role": "user",
                        "content": user_message
                    }
                ]
            }
        )
        
        # Check if request was successful
        if response.status_code == 200:
            response_data = response.json()
            return response_data['choices'][0]['message']['content']
        else:
            print(f"Error: OpenRouter API returned status code {response.status_code}")
            print(f"Response: {response.text}")
            return ''
            
    except Exception as e:
        print(f"Error making OpenRouter API request: {str(e)}")
        return ''

'There is no single agreed upon meaning of life. It\'s a question that philosophers, theologians, scientists and thinkers across cultures have grappled with for centuries. Some perspectives on potential meanings include:\n\n- Finding happiness, connection and fulfillment through relationships, experiences and self-actualization.\n\n- Reproduction and perpetuating the survival of our species.\n\n- Seeking knowledge, understanding and making sense of the universe around us.\n\n- Living ethically, reducing suffering and making the world a better place. \n\n- Discovering one\'s unique talents and purpose and living authentically.\n\n- Achieving transcendence, spiritual enlightenment or unity with a higher power/force.\n\n- Creating - whether that\'s works of art, scientific discoveries, businesses or children.\n\nUltimately, the meaning of life is a highly personal question and the "answer" may be different for each individual based on their beliefs, values and life experience. There may n

In [16]:
def iterative_marker_analysis_openrouter(major_cluster_info, marker, comma_separated_genes, annotation_history, num_iterations=2, model="anthropic/claude-3-sonnet"):
    """
    Perform iterative marker analysis using OpenRouter API.
    
    Args:
        major_cluster_info (str): Information about the cluster
        marker (DataFrame): Marker gene expression data
        comma_separated_genes (str): List of genes as comma-separated string
        annotation_history (str): Previous annotation history
        num_iterations (int): Maximum number of iterations
        model (str): OpenRouter model identifier
        
    Returns:
        tuple: (final_response_text, messages)
    """
    messages = [{"role": "user", "content": prompt_hypothesis_generator3(major_cluster_info, comma_separated_genes, annotation_history)}]

    for iteration in range(num_iterations):
        try:
            response = requests.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {os.environ.get('OPENROUTER_API_KEY')}",
                    "HTTP-Referer": "https://localhost:5000",
                    "Content-Type": "application/json"
                },
                json={
                    "model": model,
                    "temperature": 0,
                    "max_tokens": 7000,
                    "messages": messages
                }
            )
            
            # Check if request was successful
            if response.status_code == 200:
                response_data = response.json()
                conversation = response_data['choices'][0]['message']['content']

                # Check for completion
                if "FINAL ANNOTATION COMPLETED" in conversation:
                    print(f"Final annotation completed in iteration {iteration + 1}.")
                    return conversation, messages

                # Extract gene lists and get marker info
                gene_lists = re.findall(r'<check_genes>\s*(.*?)\s*</check_genes>', conversation, re.DOTALL)
                all_genes = [gene.strip() for gene_list in gene_lists for gene in gene_list.split(',')]
                unique_genes = sorted(set(all_genes))

                retrived_marker_info = get_marker_info(unique_genes, marker)
                
                # Append messages
                messages.append({"role": "assistant", "content": conversation})
                messages.append({"role": "user", "content": retrived_marker_info})

                print(f"Iteration {iteration + 1} completed.")
            else:
                print(f"Error: OpenRouter API returned status code {response.status_code}")
                print(f"Response: {response.text}")
                return '', messages

        except Exception as e:
            print(f"Error in iteration {iteration + 1}: {str(e)}")
            return '', messages

    # Final response if max iterations reached
    try:
        final_response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {os.environ.get('OPENROUTER_API_KEY')}",
                "HTTP-Referer": "https://localhost:5000",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "temperature": 0,
                "max_tokens": 7000,
                "messages": messages
            }
        )
        
        if final_response.status_code == 200:
            final_data = final_response.json()
            return final_data['choices'][0]['message']['content'], messages
        else:
            print(f"Error getting final response: {final_response.status_code}")
            print(f"Response: {final_response.text}")
            return '', messages
            
    except Exception as e:
        print(f"Error in final response: {str(e)}")
        return '', messages

In [1]:
os.environ.get('OPENROUTER_API_KEY')

NameError: name 'os' is not defined

In [21]:
import pandas as pd
import os
import requests
import re
os.environ['OPENROUTER_API_KEY'] = "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"
# 1. Create sample marker data
def create_sample_marker_data():
    """Create a sample marker DataFrame simulating gene expression data"""
    data = {
        'Gene': ['CD3E', 'CD4', 'CD8A', 'FOXP3', 'IL2RA', 'CD19', 'CD20', 'CD14', 'CD68', 'IL7R'],
        'Expression': [2.5, 1.8, 1.2, 0.8, 1.5, 0.2, 0.1, 0.3, 0.2, 1.6],
        'p_val': [0.001, 0.002, 0.001, 0.005, 0.003, 0.1, 0.2, 0.15, 0.18, 0.002],
        'avg_log2FC': [1.5, 1.2, 0.9, 0.6, 1.1, -0.2, -0.3, -0.1, -0.2, 1.3]
    }
    return pd.DataFrame(data)

# 2. Create helper function for marker info
def get_marker_info(genes, marker_df):
    """Get expression information for requested genes"""
    result = []
    for gene in genes:
        gene_data = marker_df[marker_df['Gene'] == gene]
        if not gene_data.empty:
            info = (f"Gene: {gene}\n"
                   f"Expression: {gene_data['Expression'].iloc[0]:.2f}\n"
                   f"p-value: {gene_data['p_val'].iloc[0]:.3f}\n"
                   f"log2FC: {gene_data['avg_log2FC'].iloc[0]:.2f}\n")
            result.append(info)
    return "\n".join(result)

# 3. Create prompt generator function
def prompt_hypothesis_generator3(major_cluster_info, comma_separated_genes, annotation_history):
    """Generate prompt for marker analysis"""
    return f"""
    Analyze this cell cluster based on marker gene expression:
    
    Cluster Info: {major_cluster_info}
    Marker Genes: {comma_separated_genes}
    Previous Annotation History: {annotation_history}
    
    Please analyze these markers and suggest cell type hypotheses. 
    Use <check_genes>gene1,gene2,gene3</check_genes> to request expression data for specific genes.
    
    Provide your analysis in a step-by-step format:
    1. Initial hypothesis based on key markers
    2. Request additional marker information
    3. Refine hypothesis
    4. Final cell type determination
    
    End your analysis with "FINAL ANNOTATION COMPLETED" when you reach a conclusion.
    """

# 4. Test setup and execution
def test_marker_analysis():
    # Set up test data
    marker_df = create_sample_marker_data()
    
    # Test parameters
    major_cluster_info = "Blood sample from healthy donor"
    comma_separated_genes = "CD3E,CD4,CD8A,FOXP3,IL2RA"
    annotation_history = "Initial analysis suggests muscle cell population"
    
    # Set OpenRouter API key

    try:
        # Run analysis
        final_response, messages = iterative_marker_analysis_openrouter(
            major_cluster_info=major_cluster_info,
            marker=marker_df,
            comma_separated_genes=comma_separated_genes,
            annotation_history=annotation_history,
            model="anthropic/claude-3-sonnet",
            num_iterations=2
        )
        
        # Print results
        print("\nFinal Response:")
        print("-" * 50)
        print(final_response)
        
        print("\nMessage History:")
        print("-" * 50)
        for i, msg in enumerate(messages):
            print(f"Message {i+1} ({msg['role']}):")
            print(msg['content'])
            print("-" * 30)
            
    except Exception as e:
        print(f"Error during test: {str(e)}")

# Run the test
if __name__ == "__main__":
    test_marker_analysis()

Final annotation completed in iteration 1.

Final Response:
--------------------------------------------------
1. Initial hypothesis based on key markers:
The provided marker genes (CD3E, CD4, CD8A, FOXP3, IL2RA) suggest that this cell cluster is likely a population of T cells from the blood sample. Specifically:

- CD3E is a marker for T cells
- CD4 and CD8A are markers for helper T cells and cytotoxic T cells, respectively
- FOXP3 is a marker for regulatory T cells (Tregs)
- IL2RA (CD25) is a marker for activated T cells

However, to confirm this hypothesis, I would like to check the expression of a few additional markers.

<check_genes>CD3D,CD3G,TRAC</check_genes>

2. Request additional marker information:
The additional markers (CD3D, CD3G, TRAC) are all components of the T cell receptor (TCR) complex, which is specifically expressed on T cells. High expression of these genes would strongly support the hypothesis that this cluster represents T cells.

3. Refine hypothesis:
Based on